In [106]:
from torchaudio.models.decoder import ctc_decoder, cuda_ctc_decoder
import numpy as np 
import torch
from torch.nn.functional import softmax

In [107]:
language_model_path = "/data2/brain2text/lm/languageModel/"
lexicon_phonemes_file = f"{language_model_path}lexicon_phonemes.txt"
units_txt_file_pytorch = f"{language_model_path}units_pytorch.txt"

units_txt_file_pytorch_char = f"{language_model_path}units_pytorch_character.txt"
lexicon_char_file= f"{language_model_path}lexicon_char.txt"

In [108]:
import pandas as pd
val_transcripts = pd.read_pickle("/data2/brain2text/b2t_24/transcripts_val.pkl")

In [109]:
decoder_phoneme = ctc_decoder(tokens=units_txt_file_pytorch, lexicon=lexicon_phonemes_file, 
                       beam_size=50, nbest=20, lm=None)
decoder_char = ctc_decoder(tokens=units_txt_file_pytorch_char, lexicon=lexicon_char_file, 
                       beam_size=50, nbest=20, lm=None)



[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached l

In [129]:
from torchaudio.models.decoder import download_pretrained_files

files = download_pretrained_files("librispeech-4-gram")

decoder_char_lm = ctc_decoder(tokens=units_txt_file_pytorch_char, lexicon=lexicon_char_file, 
                       beam_size=50, nbest=20, lm=files.lm, lm_weight=3, word_score=-2)


In [131]:
model_logits_phone = np.load("/data2/brain2text/b2t_24/logits/tm_transformer_b2t_24+25_large_wide/logits_val.npz")
model_logits_char = np.load("/data2/brain2text/b2t_24/logits/tm_transformer_combined_lw_char/logits_val.npz")

idx = 200
print(val_transcripts[idx])


single_trial_logits_phone = torch.as_tensor(model_logits_phone[f'arr_{idx}']).float().unsqueeze(0)
beam_search_phoneme = decoder_phoneme(single_trial_logits_phone)
beam_search_transcript_phoneme= " ".join(beam_search_phoneme[0][0].words).strip()
print(beam_search_transcript_phoneme)

single_trial_logits_char = torch.as_tensor(model_logits_char[f'arr_{idx}']).float().unsqueeze(0)
beam_search_char = decoder_char(single_trial_logits_char)
beam_search_transcript_char = " ".join(beam_search_char[0][0].words).strip()
print(beam_search_transcript_char)

beam_search_char_lm = decoder_char_lm(single_trial_logits_char)
beam_search_transcript_char_lm = " ".join(beam_search_char_lm[0][0].words).strip()
print(beam_search_transcript_char_lm)


i say that but yet i know too
aye sci that butt yet eye nau thuy
i see that bit yet i know too
i see that but yet i know too


In [147]:
model_logits_phone = np.load("/data2/brain2text/b2t_24/logits/tm_transformer_b2t_24+25_large_wide/logits_val.npz")
model_logits_char = np.load("/data2/brain2text/b2t_24/logits/tm_transformer_combined_lw_char/logits_val.npz")

idx = 250
print("Transcript: ", val_transcripts[idx])


print("\n")
single_trial_logits_phone = torch.as_tensor(model_logits_phone[f'arr_{idx}']).float().unsqueeze(0)
beam_search_phoneme = decoder_phoneme(single_trial_logits_phone)
for i in range(20):
    beam_search_transcript_phoneme= " ".join(beam_search_phoneme[0][i].words).strip()
    print(beam_search_transcript_phoneme)

print("\n")
single_trial_logits_char = torch.as_tensor(model_logits_char[f'arr_{idx}']).float().unsqueeze(0)
beam_search_char = decoder_char(single_trial_logits_char)
for i in range(20):
    beam_search_transcript_char = " ".join(beam_search_char[0][i].words).strip()
    print(beam_search_transcript_char)




Transcript:  let's see about getting parents more involved


that's seery about getting pinning mohr involved
that's seery about getting pinning morr involved
that's seery about getting pinning more involved
that's seery about getting pinning mor involved
that's seery about getting pinning mohr involve
that's seery about getting pinning morr involve
that's seery about getting pinning mor involve
that's seery about getting pinning more involve
that's seery about getting penning morr involved
that's seery about getting penning more involved


that's hey about get counting more involved
that's hey about gets counting more involved
that's se about get counting more involved
that's see about get counting more involved
that's se about gets counting more involved
that's hey about getts counting more involved
that's see about gets counting more involved
that's hey about get touting more involved
that's hey about get counting more involve d
that's hey about gets counting more involve d


In [101]:

single_trial_logits_char = torch.as_tensor(model_logits_char[f'arr_{idx}']).float().unsqueeze(0)


In [56]:
start = time.time()

end = time.time()
print(end-start)

0.013104915618896484


In [16]:
single_trial_logits.shape

torch.Size([1, 61, 41])

In [15]:
decoder.decode_begin()
for t in range(32):
    decoder.decode_step(single_trial_logits[0, t:t + 1, :])
    
decoder.decode_end()
beam_search_result_inc = decoder.get_final_hypothesis()
print(beam_search_result_inc)

[CTCHypothesis(tokens=tensor([40, 35, 26,  9, 40,  2, 14, 31, 12, 40]), words=['VOID', 'AFTER'], score=522.8263626098633, timesteps=tensor([ 0, 10, 12, 14, 17, 22, 24, 25, 27, 31], dtype=torch.int32)), CTCHypothesis(tokens=tensor([40, 35, 26,  9, 40, 18, 40,  2, 14, 31, 12, 40]), words=['VOID', 'E.', 'AFTER'], score=520.3375873565674, timesteps=tensor([ 0, 10, 12, 14, 16, 17, 18, 22, 24, 25, 27, 31], dtype=torch.int32)), CTCHypothesis(tokens=tensor([40, 35, 26,  9, 40, 18, 40,  2, 14, 31, 12, 40]), words=['VOID', 'E', 'AFTER'], score=520.3375873565674, timesteps=tensor([ 0, 10, 12, 14, 16, 17, 18, 22, 24, 25, 27, 31], dtype=torch.int32)), CTCHypothesis(tokens=tensor([40, 35, 26,  9, 40, 18, 40,  2, 14, 31, 12, 40]), words=['VOID', 'EE', 'AFTER'], score=520.3375873565674, timesteps=tensor([ 0, 10, 12, 14, 16, 17, 18, 22, 24, 25, 27, 31], dtype=torch.int32)), CTCHypothesis(tokens=tensor([40, 35, 26,  9, 40,  2, 14, 31, 12, 23, 34, 23, 40]), words=['VOID', 'AFTERNOON'], score=519.06853389

In [59]:
import heapq

def make_new_beam():
  fn = lambda : (NEG_INF, NEG_INF)
  return collections.defaultdict(fn)

def logsumexp(*args):
  """
  Stable log sum exp.
  (Optimized version)
  """
  if not args:
      return NEG_INF

  # <-- 2. Optimized logsumexp
  # Find max in one pass
  a_max = args[0]
  for a in args[1:]:
      if a > a_max:
          a_max = a

  if a_max == NEG_INF:
      return NEG_INF

  # Calculate sum in a second pass
  lsp_sum = 0.0
  for a in args:
      lsp_sum += math.exp(a - a_max)
      
  return a_max + math.log(lsp_sum)

def decode(probs, beam_size=100, blank=0):
  """
  Performs inference for the given output probabilities.
  Arguments:
      probs: The output probabilities (e.g. post-softmax) for each
        time step. Should be an array of shape (time x output dim).
      beam_size (int): Size of the beam to use during inference.
      blank (int): Index of the CTC blank label.
  Returns the output label sequence and the corresponding negative
  log-likelihood estimated by the decoder.
  """
  T, S = probs.shape
  # Convert to log-probabilities once at the beginning
  log_probs = np.log(probs)

  # Elements in the beam are (prefix, (p_blank, p_no_blank))
  # Initialize the beam with the empty sequence, a probability of
  # 1 for ending in blank and zero for ending in non-blank
  # (in log space).
  beam = [(tuple(), (0.0, NEG_INF))]

  for t in range(T): # Loop over time

    # A default dictionary to store the next step candidates.
    next_beam = make_new_beam()

    # <-- 3. Pre-slice probabilities for this time step
    log_probs_t = log_probs[t, :]

    for s in range(S): # Loop over vocab
      p = log_probs_t[s] # Use the pre-sliced array

      # The variables p_b and p_nb are respectively the
      # probabilities for the prefix given that it ends in a
      # blank and does not end in a blank at this time step.
      for prefix, (p_b, p_nb) in beam: # Loop over beam

        # If we propose a blank the prefix doesn't change.
        # Only the probability of ending in blank gets updated.
        if s == blank:
          n_p_b, n_p_nb = next_beam[prefix]
          n_p_b = logsumexp(n_p_b, p_b + p, p_nb + p)
          next_beam[prefix] = (n_p_b, n_p_nb)
          continue

        # Extend the prefix by the new character s and add it to
        # the beam. Only the probability of not ending in blank
        # gets updated.
        end_t = prefix[-1] if prefix else None
        n_prefix = prefix + (s,)
        n_p_b, n_p_nb = next_beam[n_prefix]
        if s != end_t:
          n_p_nb = logsumexp(n_p_nb, p_b + p, p_nb + p)
        else:
          # We don't include the previous probability of not ending
          # in blank (p_nb) if s is repeated at the end. The CTC
          # algorithm merges characters not separated by a blank.
          n_p_nb = logsumexp(n_p_nb, p_b + p)
          
        next_beam[n_prefix] = (n_p_b, n_p_nb)

        # If s is repeated at the end we also update the unchanged
        # prefix. This is the merging case.
        if s == end_t:
          n_p_b, n_p_nb = next_beam[prefix]
          n_p_nb = logsumexp(n_p_nb, p_nb + p)
          next_beam[prefix] = (n_p_b, n_p_nb)

    # <-- 4. MAJOR OPTIMIZATION: Use heapq.nlargest instead of full sort
    # Original:
    # beam = sorted(next_beam.items(),
    #         key=lambda x : logsumexp(*x[1]),
    #         reverse=True)
    # beam = beam[:beam_size]
    
    # Optimized:
    beam_items = next_beam.items()
    beam = heapq.nlargest(beam_size,
                          beam_items,
                          key=lambda x: logsumexp(*x[1]))

  # Final selection
  best = beam[0]
  return best[0], -logsumexp(*best[1])

if __name__ == "__main__":
  np.random.seed(3)

  time = 50
  output_dim = 20

  probs = np.random.rand(time, output_dim)
  probs = probs / np.sum(probs, axis=1, keepdims=True)

  labels, score = decode(probs)
  print(labels)
  print("Score {:.3f}".format(score))

(5, 12, 8, 16, 5, 13, 19, 7, 19, 3, 2, 4, 11, 19, 8, 14, 18, 2, 13, 17, 14, 18, 5, 1, 6, 9)
Score 97.032


In [63]:
from scipy.special import softmax
logits = softmax(model_logits[f'arr_750}'],axis=1)
import time

In [65]:
start = time.time()
labels, probs = decode(logits)
end = time.time()
print(end-start)

0.6410787105560303
